In [1]:
import pandas as pd
import numpy as np


In [4]:
log = pd.read_csv('dataset/EClog HTTP_level e_commerce data based on server access logs for an online store/eclog_1day.csv',header=0,index_col=None)
log = log.loc[:,"Uri"]

In [5]:
print(log.head())

0    /inne/informacja_online.php
1    /inne/informacja_online.php
2      /wyszukiwanie-query1.html
3    /inne/informacja_online.php
4    /inne/informacja_online.php
Name: Uri, dtype: object


In [4]:
# print(log.loc[:,'Uri'].unique().size)
# print(log.size)

In [5]:
# log.loc[:,'Uri'].to_csv('dataset/eclog_uri.csv')

In [6]:
# log = pd.read_csv('dataset/eclog_uri.csv',header=0,index_col=0)

In [7]:
print(log)

                                                        Uri
0                                              /p-9710.html
1         /szablony/shop_11.rwd/css/style.php?ncss=style...
2                                   /javascript/skrypty.php
3                                   /javascript/produkt.php
4            /szablony/shop_11.rwd/obrazki/szablon/info.png
...                                                     ...
35157686                        /inne/informacja_online.php
35157687                       /wyszukiwanie-query6283.html
35157688                             /images/image11466.jpg
35157689                   /images/mini/250px_image9012.jpg
35157690                        /inne/informacja_online.php

[35157691 rows x 1 columns]


In [8]:
tmp = []
max = 0
for i,data in enumerate(log.loc[:,'Uri']):
    data = data.split("/")
    if len(data)>max:
        max= len(data)
    tmp.append(data)
    


In [9]:
for i in range(10):
    print(tmp[i][1:])

['p-9710.html']
['szablony', 'shop_11.rwd', 'css', 'style.php?ncss=style,boxy,moduly,produkt']
['javascript', 'skrypty.php']
['javascript', 'produkt.php']
['szablony', 'shop_11.rwd', 'obrazki', 'szablon', 'info.png']
['szablony', 'shop_11.rwd', 'obrazki', 'ikony', 'gwiazdka.png']
['images', 'mini', '250px_image1.jpg']
['inne', 'informacja_online.php']
['inne', 'informacja_online.php']
['images', 'mini', '70px_image2.jpg']


In [10]:
data = np.zeros((35157691,max))

In [11]:
voca=[]

In [12]:
from tqdm.notebook import tqdm
for s_i,sentence in tqdm(enumerate(tmp)):
    for w_i, word in enumerate(sentence):
        try: 
            voca_idx = voca.index(word)
        except ValueError:
            voca_idx = -1
        if voca_idx==-1:
            voca.append(word)
            voca_idx= len(voca)-1
        data[s_i,w_i]=voca_idx

0it [00:00, ?it/s]

In [ ]:
for i in range(10):
    print(data[i])

In [16]:
import pickle

with open("./voca.pickle","wb") as fw:
    pickle.dump(voca,fw,protocol=4)
with open("./data.pickle","wb") as dw:
    pickle.dump(data,dw,protocol=4)


In [20]:
np.save('data.npy',data)

In [19]:
print(len(voca))


591183


In [23]:
import torch
import torch.nn as nn
from torch.utils.data import dataloader,dataset

class Dataset(dataset.Dataset):
    def __init__(self,data):
        super(Dataset,self).__init__()
        self.input = data
        self.output = data
    def __getitem__(self,index):
        return torch.LongTensor(self.input[index]),torch.LongTensor(self.output[index])
    def __len__(self):
        return len(self.input)
train_set = Dataset(data)
train_loader = dataloader.DataLoader(train_set,batch_size=512,shuffle=True)

In [ ]:
class Model(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(Model,self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.output_size = output_size
        self.encode = nn.Sequential(
            nn.Embedding(input_size,hidden_size),
            nn.GRU(hidden_size,output_size)
        )
        self.decode = nn.Sequential(
            nn.GRU(output_size,hidden_size)
        )